In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import os

### Retrieve games and team names using NBA API

In [2]:
nba_teams = teams.get_teams()
gamefinder = leaguegamefinder.LeagueGameFinder()
games = gamefinder.get_data_frames()[0]
games.head()

total_df = pd.DataFrame()
print(games.columns)

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')


In [3]:
team_ids = []
team_names = []
for team in nba_teams:
    team_names.append((team['id'], team['abbreviation']))

### Convert GAME_DATE column to date_time

In [4]:
games.GAME_DATE = pd.to_datetime(games.GAME_DATE)

print(games.GAME_DATE.min())
print(games.GAME_DATE.max())

2014-11-22 00:00:00
2021-04-25 00:00:00


### Filter games and find shooting stats against each team

In [5]:
non_bubble_games = games[(games.GAME_DATE < '2020-07-01') | (games.GAME_DATE > '2020-10-20')]
for (id, name) in team_names:
    games_against_team = non_bubble_games[non_bubble_games.MATCHUP.str.contains(name)]
    games_against_team = games_against_team[games_against_team['TEAM_ABBREVIATION'] != name]
    average_against = games_against_team.mean(axis = 0)
    team_vals = pd.DataFrame()
    team_vals = average_against.filter(items = ['FG_PCT', 'FG3_PCT', 'FT_PCT'])
    team_vals['TEAM'] = name
    total_df = total_df.append(team_vals, ignore_index = True)

total_df

C:\Users\gourn\anaconda3\envs\dva\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """


,FG3_PCT,FG_PCT,FT_PCT,TEAM
0,0.341966,0.446629,0.772811,ATL
1,0.335966,0.444065,0.759824,BOS
2,0.353636,0.464939,0.759152,CLE
3,0.354466,0.460037,0.770250,NOP
4,0.346930,0.451781,0.766769,CHI
5,0.352025,0.456760,0.766557,DAL
6,0.352237,0.462678,0.756897,DEN
7,0.342478,0.441065,0.762289,GSW
8,0.347471,0.460848,0.755003,HOU
9,0.346285,0.448265,0.758990,LAC


In [6]:
stats = total_df

### Append state to average stats DataFrame

In [7]:
datapath = os.getcwd()
states = pd.read_csv(datapath + '/state_teams.csv')
states.set_index('TEAM')

,State
TEAM,
ATL,Georgia
BOS,Massachusetts
CLE,Ohio
NOP,Louisiana
CHI,Illinois
DAL,Texas
DEN,Colorado
GSW,California
HOU,Texas


In [8]:
data = states.join(stats.set_index('TEAM'), on='TEAM')

data.set_index('TEAM')

,State,FG3_PCT,FG_PCT,FT_PCT
TEAM,,,,
ATL,Georgia,0.341966,0.446629,0.772811
BOS,Massachusetts,0.335966,0.444065,0.759824
CLE,Ohio,0.353636,0.464939,0.759152
NOP,Louisiana,0.354466,0.460037,0.770250
CHI,Illinois,0.346930,0.451781,0.766769
DAL,Texas,0.352025,0.456760,0.766557
DEN,Colorado,0.352237,0.462678,0.756897
GSW,California,0.342478,0.441065,0.762289
HOU,Texas,0.347471,0.460848,0.755003


In [9]:
data.to_csv('data.csv', index='False')

### Extract bubble games and find average stats in bubble

In [10]:
games_bubble = games[(games.GAME_DATE > '2020-07-01') & (games.SEASON_ID.str[-4:] == '2019')]

In [11]:
bubble_df = pd.DataFrame()
for (id, name) in team_names:
    games_against_team = games_bubble[games_bubble.MATCHUP.str.contains(name)]
    games_against_team = games_against_team[games_against_team['TEAM_ABBREVIATION'] != name]
    average_against = games_against_team.mean(axis = 0)
    team_vals = pd.DataFrame()
    team_vals = average_against.filter(items = ['FG_PCT', 'FG3_PCT', 'FT_PCT'])
    team_vals['TEAM'] = name
    bubble_df = bubble_df.append(team_vals, ignore_index = True)

C:\Users\gourn\anaconda3\envs\dva\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """


In [12]:
bubble_df.dropna()
bubble_avgs = bubble_df.mean(axis=0)

bubble_avgs = bubble_avgs.reset_index()
bubble_avgs = bubble_avgs.rename(columns={0: 'value', 'index': 'stat'}).set_index('stat')

bubble_avgs

,value
stat,
FG3_PCT,0.360107
FG_PCT,0.459553
FT_PCT,0.786202


In [13]:
bubble_avgs.to_csv('bubble_data.csv', index='False')